# Day 4: Deep Hedging Basics

## Learning Objectives
- Understand the deep hedging framework
- Implement neural network for learning hedge ratios
- Compare deep hedging to Black-Scholes delta
- Analyze performance with transaction costs

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from dataclasses import dataclass
from typing import Tuple, List, Optional
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

np.random.seed(42)
torch.manual_seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Deep Hedging Framework

### Traditional vs Deep Hedging

**Traditional (Black-Scholes) Hedging:**
- Assumes specific model (GBM, constant volatility)
- Derives hedge ratio analytically: $\delta = N(d_1)$
- Ignores transaction costs in derivation

**Deep Hedging:**
- Learn hedge ratios directly from data
- Can incorporate transaction costs in training
- Model-agnostic: works with any price dynamics

### The Deep Hedging Objective

We want to find a hedging strategy $\delta_t = f_\theta(\text{features}_t)$ that minimizes:

$$\mathcal{L} = \mathbb{E}[\rho(\text{Hedging P\&L})]$$

Where $\rho$ is a risk measure (e.g., variance, CVaR, or entropic risk).

The hedging P&L for a short option position is:
$$\text{P\&L} = V_0 + \sum_{t=0}^{T-1} \delta_t (S_{t+1} - S_t) - c|\delta_t - \delta_{t-1}|S_t - \text{Payoff}_T$$

Where $c$ is the proportional transaction cost.

In [ ]:
# Black-Scholes functions for comparison
class BlackScholes:
    """Black-Scholes model for benchmarking."""
    
    @staticmethod
    def d1(S, K, T, r, sigma):
        if T <= 0:
            return np.where(S > K, np.inf, -np.inf)
        return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    @staticmethod
    def call_price(S, K, T, r, sigma):
        if T <= 0:
            return np.maximum(S - K, 0)
        d1 = BlackScholes.d1(S, K, T, r, sigma)
        d2 = d1 - sigma * np.sqrt(T)
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    @staticmethod
    def delta(S, K, T, r, sigma):
        if T <= 0:
            return np.where(S > K, 1.0, 0.0)
        d1 = BlackScholes.d1(S, K, T, r, sigma)
        return norm.cdf(d1)


bs = BlackScholes()

## 2. Data Generation

Generate simulated stock price paths for training the hedging network.

In [ ]:
class DataGenerator:
    """Generate training data for deep hedging."""
    
    def __init__(self, S0: float, K: float, T: float, r: float, sigma: float, n_steps: int):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n_steps = n_steps
        self.dt = T / n_steps
    
    def generate_paths(self, n_paths: int) -> Tuple[np.ndarray, np.ndarray]:
        """
        Generate GBM price paths.
        
        Returns:
            prices: (n_paths, n_steps + 1) array
            times: (n_steps + 1,) array
        """
        times = np.linspace(0, self.T, self.n_steps + 1)
        
        # Generate random increments
        Z = np.random.standard_normal((n_paths, self.n_steps))
        
        # Log returns under risk-neutral measure
        log_returns = (self.r - 0.5 * self.sigma**2) * self.dt + self.sigma * np.sqrt(self.dt) * Z
        
        # Build price paths
        log_prices = np.zeros((n_paths, self.n_steps + 1))
        log_prices[:, 0] = np.log(self.S0)
        log_prices[:, 1:] = np.log(self.S0) + np.cumsum(log_returns, axis=1)
        
        prices = np.exp(log_prices)
        return prices, times
    
    def get_features(self, S: np.ndarray, t: float) -> np.ndarray:
        """
        Create features for the hedging network.
        
        Features:
            - Log-moneyness: log(S/K)
            - Time to maturity: T - t
            - Normalized price: S / K
        """
        tau = self.T - t
        log_moneyness = np.log(S / self.K)
        
        features = np.column_stack([
            log_moneyness,
            np.full(len(S), tau),
            S / self.K
        ])
        return features


# Generate sample data
params = {
    'S0': 100,
    'K': 100,
    'T': 0.25,  # 3 months
    'r': 0.05,
    'sigma': 0.2,
    'n_steps': 63  # Daily rebalancing for ~3 months
}

data_gen = DataGenerator(**params)
sample_prices, times = data_gen.generate_paths(n_paths=5)

plt.figure(figsize=(12, 5))
for i in range(5):
    plt.plot(times * 252, sample_prices[i], alpha=0.7)
plt.axhline(y=100, color='red', linestyle='--', label='Strike')
plt.xlabel('Trading Days')
plt.ylabel('Stock Price ($)')
plt.title('Sample Price Paths for Training')
plt.legend()
plt.show()

## 3. Neural Network for Hedging

The hedging network takes market features as input and outputs the hedge ratio (delta).

In [ ]:
class HedgingNetwork(nn.Module):
    """Neural network for learning hedge ratios."""
    
    def __init__(self, input_dim: int = 3, hidden_dims: List[int] = [32, 32]):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidden_dim))
            prev_dim = hidden_dim
        
        # Output layer: sigmoid to constrain delta to [0, 1] for call options
        layers.append(nn.Linear(prev_dim, 1))
        layers.append(nn.Sigmoid())
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x).squeeze(-1)


# Test network
model = HedgingNetwork(input_dim=3, hidden_dims=[32, 32])
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

## 4. Deep Hedging Training

### Loss Function: Hedging P&L Variance

We minimize the variance of the hedging P&L to find robust strategies.

In [ ]:
class DeepHedger:
    """Deep hedging trainer and evaluator."""
    
    def __init__(self, S0: float, K: float, T: float, r: float, sigma: float,
                 n_steps: int, transaction_cost: float = 0.0):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n_steps = n_steps
        self.dt = T / n_steps
        self.transaction_cost = transaction_cost
        
        self.data_gen = DataGenerator(S0, K, T, r, sigma, n_steps)
        self.model = HedgingNetwork(input_dim=3, hidden_dims=[64, 64, 32]).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        
        # Initial option premium
        self.V0 = BlackScholes.call_price(S0, K, T, r, sigma)
    
    def compute_pnl(self, prices: torch.Tensor, deltas: torch.Tensor) -> torch.Tensor:
        """
        Compute hedging P&L for a batch of paths.
        
        Args:
            prices: (batch_size, n_steps + 1) tensor
            deltas: (batch_size, n_steps) tensor of hedge ratios
        
        Returns:
            pnl: (batch_size,) tensor
        """
        batch_size = prices.shape[0]
        
        # Stock returns
        price_changes = prices[:, 1:] - prices[:, :-1]  # (batch, n_steps)
        
        # Hedging gains
        hedge_gains = torch.sum(deltas * price_changes, dim=1)  # (batch,)
        
        # Transaction costs
        delta_changes = torch.zeros_like(deltas)
        delta_changes[:, 0] = deltas[:, 0]  # Initial position
        delta_changes[:, 1:] = deltas[:, 1:] - deltas[:, :-1]
        
        costs = self.transaction_cost * torch.sum(
            torch.abs(delta_changes) * prices[:, :-1], dim=1
        )
        
        # Final payoff (short call)
        payoff = torch.clamp(prices[:, -1] - self.K, min=0)
        
        # Total P&L: received premium + hedge gains - costs - payoff owed
        pnl = self.V0 + hedge_gains - costs - payoff
        
        return pnl
    
    def compute_loss(self, pnl: torch.Tensor, risk_aversion: float = 1.0) -> torch.Tensor:
        """
        Compute loss based on mean-variance criterion.
        
        Loss = -E[PnL] + λ * Var[PnL]
        """
        mean_pnl = torch.mean(pnl)
        var_pnl = torch.var(pnl)
        return -mean_pnl + risk_aversion * var_pnl
    
    def train_epoch(self, n_paths: int = 10000, batch_size: int = 256) -> float:
        """Train for one epoch."""
        self.model.train()
        
        # Generate training data
        prices, times = self.data_gen.generate_paths(n_paths)
        prices_tensor = torch.FloatTensor(prices).to(device)
        
        total_loss = 0
        n_batches = n_paths // batch_size
        
        for i in range(n_batches):
            start_idx = i * batch_size
            end_idx = start_idx + batch_size
            batch_prices = prices_tensor[start_idx:end_idx]
            
            # Compute deltas for each time step
            deltas = []
            for t in range(self.n_steps):
                S_t = batch_prices[:, t].cpu().numpy()
                features = self.data_gen.get_features(S_t, times[t])
                features_tensor = torch.FloatTensor(features).to(device)
                delta_t = self.model(features_tensor)
                deltas.append(delta_t)
            
            deltas = torch.stack(deltas, dim=1)  # (batch, n_steps)
            
            # Compute P&L and loss
            pnl = self.compute_pnl(batch_prices, deltas)
            loss = self.compute_loss(pnl)
            
            # Backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
        
        return total_loss / n_batches
    
    def evaluate(self, n_paths: int = 5000) -> dict:
        """Evaluate the trained model."""
        self.model.eval()
        
        prices, times = self.data_gen.generate_paths(n_paths)
        prices_tensor = torch.FloatTensor(prices).to(device)
        
        with torch.no_grad():
            # Deep hedging deltas
            deep_deltas = []
            for t in range(self.n_steps):
                S_t = prices[:, t]
                features = self.data_gen.get_features(S_t, times[t])
                features_tensor = torch.FloatTensor(features).to(device)
                delta_t = self.model(features_tensor)
                deep_deltas.append(delta_t.cpu().numpy())
            
            deep_deltas = np.array(deep_deltas).T  # (n_paths, n_steps)
            
            # Black-Scholes deltas
            bs_deltas = np.zeros((n_paths, self.n_steps))
            for t in range(self.n_steps):
                tau = self.T - times[t]
                bs_deltas[:, t] = BlackScholes.delta(prices[:, t], self.K, tau, self.r, self.sigma)
            
            # Compute P&Ls
            deep_pnl = self.compute_pnl(
                prices_tensor, 
                torch.FloatTensor(deep_deltas).to(device)
            ).cpu().numpy()
            
            bs_pnl = self.compute_pnl(
                prices_tensor,
                torch.FloatTensor(bs_deltas).to(device)
            ).cpu().numpy()
        
        return {
            'prices': prices,
            'times': times,
            'deep_deltas': deep_deltas,
            'bs_deltas': bs_deltas,
            'deep_pnl': deep_pnl,
            'bs_pnl': bs_pnl
        }

In [ ]:
# Train deep hedger without transaction costs first
print("Training Deep Hedger (No Transaction Costs)...")
print("=" * 50)

hedger_no_cost = DeepHedger(
    S0=100, K=100, T=0.25, r=0.05, sigma=0.2,
    n_steps=63, transaction_cost=0.0
)

losses = []
n_epochs = 50

for epoch in range(n_epochs):
    loss = hedger_no_cost.train_epoch(n_paths=5000, batch_size=256)
    losses.append(loss)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1:3d}: Loss = {loss:.6f}")

plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss (No Transaction Costs)')
plt.show()

In [ ]:
# Evaluate without transaction costs
results_no_cost = hedger_no_cost.evaluate(n_paths=5000)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Compare delta predictions for sample paths
ax1 = axes[0, 0]
sample_idx = 0
ax1.plot(results_no_cost['times'][:-1] * 252, results_no_cost['deep_deltas'][sample_idx], 
         'b-', label='Deep Hedge', linewidth=2)
ax1.plot(results_no_cost['times'][:-1] * 252, results_no_cost['bs_deltas'][sample_idx], 
         'r--', label='BS Delta', linewidth=2)
ax1.set_xlabel('Trading Days')
ax1.set_ylabel('Delta')
ax1.set_title('Delta Comparison (Sample Path)')
ax1.legend()
ax1.set_ylim(-0.05, 1.05)

# Delta vs Price scatter
ax2 = axes[0, 1]
mid_time_idx = 30
ax2.scatter(results_no_cost['prices'][:, mid_time_idx], 
            results_no_cost['deep_deltas'][:, mid_time_idx], 
            alpha=0.3, s=10, label='Deep Hedge')
# BS delta curve
prices_sorted = np.sort(results_no_cost['prices'][:, mid_time_idx])
tau_mid = 0.25 - results_no_cost['times'][mid_time_idx]
bs_curve = BlackScholes.delta(prices_sorted, 100, tau_mid, 0.05, 0.2)
ax2.plot(prices_sorted, bs_curve, 'r-', linewidth=2, label='BS Delta')
ax2.set_xlabel('Stock Price ($)')
ax2.set_ylabel('Delta')
ax2.set_title(f'Delta vs Price at t={mid_time_idx} days')
ax2.legend()

# P&L distributions
ax3 = axes[1, 0]
ax3.hist(results_no_cost['deep_pnl'], bins=50, alpha=0.6, label='Deep Hedge', density=True)
ax3.hist(results_no_cost['bs_pnl'], bins=50, alpha=0.6, label='BS Hedge', density=True)
ax3.axvline(x=0, color='black', linestyle='--')
ax3.set_xlabel('P&L ($)')
ax3.set_ylabel('Density')
ax3.set_title('P&L Distribution (No Transaction Costs)')
ax3.legend()

# P&L comparison box plot
ax4 = axes[1, 1]
bp = ax4.boxplot([results_no_cost['deep_pnl'], results_no_cost['bs_pnl']], 
                  labels=['Deep Hedge', 'BS Hedge'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
ax4.axhline(y=0, color='black', linestyle='--')
ax4.set_ylabel('P&L ($)')
ax4.set_title('P&L Comparison')

plt.tight_layout()
plt.show()

# Statistics
print("\nPerformance Comparison (No Transaction Costs)")
print("=" * 50)
print(f"{'Metric':<20} {'Deep Hedge':>15} {'BS Hedge':>15}")
print("-" * 50)
print(f"{'Mean P&L'::<20} ${np.mean(results_no_cost['deep_pnl']):>14.4f} ${np.mean(results_no_cost['bs_pnl']):>14.4f}")
print(f"{'Std P&L'::<20} ${np.std(results_no_cost['deep_pnl']):>14.4f} ${np.std(results_no_cost['bs_pnl']):>14.4f}")
print(f"{'5th Percentile'::<20} ${np.percentile(results_no_cost['deep_pnl'], 5):>14.4f} ${np.percentile(results_no_cost['bs_pnl'], 5):>14.4f}")
print(f"{'95th Percentile'::<20} ${np.percentile(results_no_cost['deep_pnl'], 95):>14.4f} ${np.percentile(results_no_cost['bs_pnl'], 95):>14.4f}")

## 5. Deep Hedging with Transaction Costs

The real advantage of deep hedging appears when we incorporate transaction costs.

In [ ]:
# Train with transaction costs
print("Training Deep Hedger (With 10bps Transaction Costs)...")
print("=" * 50)

hedger_with_cost = DeepHedger(
    S0=100, K=100, T=0.25, r=0.05, sigma=0.2,
    n_steps=63, transaction_cost=0.001  # 10bps
)

losses_cost = []

for epoch in range(n_epochs):
    loss = hedger_with_cost.train_epoch(n_paths=5000, batch_size=256)
    losses_cost.append(loss)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1:3d}: Loss = {loss:.6f}")

plt.figure(figsize=(10, 4))
plt.plot(losses_cost)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss (10bps Transaction Costs)')
plt.show()

In [ ]:
# Evaluate with transaction costs
results_with_cost = hedger_with_cost.evaluate(n_paths=5000)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Compare delta - deep hedge should be "smoother"
ax1 = axes[0, 0]
sample_idx = 0
ax1.plot(results_with_cost['times'][:-1] * 252, results_with_cost['deep_deltas'][sample_idx], 
         'b-', label='Deep Hedge (with costs)', linewidth=2)
ax1.plot(results_with_cost['times'][:-1] * 252, results_with_cost['bs_deltas'][sample_idx], 
         'r--', label='BS Delta', linewidth=2, alpha=0.7)
ax1.set_xlabel('Trading Days')
ax1.set_ylabel('Delta')
ax1.set_title('Delta Comparison with Transaction Costs')
ax1.legend()

# Delta changes comparison
ax2 = axes[0, 1]
deep_changes = np.abs(np.diff(results_with_cost['deep_deltas'], axis=1)).mean(axis=0)
bs_changes = np.abs(np.diff(results_with_cost['bs_deltas'], axis=1)).mean(axis=0)
ax2.plot(results_with_cost['times'][1:-1] * 252, deep_changes, 'b-', label='Deep Hedge', linewidth=2)
ax2.plot(results_with_cost['times'][1:-1] * 252, bs_changes, 'r--', label='BS Hedge', linewidth=2)
ax2.set_xlabel('Trading Days')
ax2.set_ylabel('Mean Absolute Delta Change')
ax2.set_title('Trading Activity Comparison')
ax2.legend()

# P&L distributions
ax3 = axes[1, 0]
ax3.hist(results_with_cost['deep_pnl'], bins=50, alpha=0.6, label='Deep Hedge', density=True)
ax3.hist(results_with_cost['bs_pnl'], bins=50, alpha=0.6, label='BS Hedge', density=True)
ax3.axvline(x=0, color='black', linestyle='--')
ax3.set_xlabel('P&L ($)')
ax3.set_ylabel('Density')
ax3.set_title('P&L Distribution (With 10bps Transaction Costs)')
ax3.legend()

# P&L comparison
ax4 = axes[1, 1]
bp = ax4.boxplot([results_with_cost['deep_pnl'], results_with_cost['bs_pnl']], 
                  labels=['Deep Hedge', 'BS Hedge'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
ax4.axhline(y=0, color='black', linestyle='--')
ax4.set_ylabel('P&L ($)')
ax4.set_title('P&L Comparison (With Costs)')

plt.tight_layout()
plt.show()

# Statistics
print("\nPerformance Comparison (With 10bps Transaction Costs)")
print("=" * 50)
print(f"{'Metric':<20} {'Deep Hedge':>15} {'BS Hedge':>15}")
print("-" * 50)
print(f"{'Mean P&L'::<20} ${np.mean(results_with_cost['deep_pnl']):>14.4f} ${np.mean(results_with_cost['bs_pnl']):>14.4f}")
print(f"{'Std P&L'::<20} ${np.std(results_with_cost['deep_pnl']):>14.4f} ${np.std(results_with_cost['bs_pnl']):>14.4f}")
print(f"{'5th Percentile'::<20} ${np.percentile(results_with_cost['deep_pnl'], 5):>14.4f} ${np.percentile(results_with_cost['bs_pnl'], 5):>14.4f}")
print(f"{'95th Percentile'::<20} ${np.percentile(results_with_cost['deep_pnl'], 95):>14.4f} ${np.percentile(results_with_cost['bs_pnl'], 95):>14.4f}")

# Trading activity
deep_total_trading = np.mean(np.sum(np.abs(np.diff(results_with_cost['deep_deltas'], axis=1)), axis=1))
bs_total_trading = np.mean(np.sum(np.abs(np.diff(results_with_cost['bs_deltas'], axis=1)), axis=1))
print(f"\n{'Avg Total Trading'::<20} {deep_total_trading:>15.4f} {bs_total_trading:>15.4f}")
print(f"{'Trading Reduction'::<20} {(1 - deep_total_trading/bs_total_trading)*100:>14.1f}%")

## 6. Visualization: Learned Delta Surface

In [ ]:
# Create delta surface comparison
hedger_with_cost.model.eval()

spot_range = np.linspace(80, 120, 50)
time_range = np.array([0.0, 0.05, 0.1, 0.15, 0.2, 0.24])

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, t in enumerate(time_range):
    ax = axes[idx // 3, idx % 3]
    tau = 0.25 - t
    
    # BS delta
    bs_deltas = BlackScholes.delta(spot_range, 100, tau, 0.05, 0.2)
    
    # Deep hedge delta
    with torch.no_grad():
        features = hedger_with_cost.data_gen.get_features(spot_range, t)
        features_tensor = torch.FloatTensor(features).to(device)
        deep_deltas = hedger_with_cost.model(features_tensor).cpu().numpy()
    
    ax.plot(spot_range, bs_deltas, 'r-', label='BS Delta', linewidth=2)
    ax.plot(spot_range, deep_deltas, 'b--', label='Deep Hedge', linewidth=2)
    ax.axvline(x=100, color='gray', linestyle=':', alpha=0.5)
    ax.set_xlabel('Spot Price ($)')
    ax.set_ylabel('Delta')
    ax.set_title(f't = {t*252:.0f} days (τ = {tau*252:.0f} days to expiry)')
    ax.legend()
    ax.set_ylim(-0.05, 1.05)

plt.tight_layout()
plt.suptitle('Delta Surface: Deep Hedging vs Black-Scholes', y=1.02, fontsize=14)
plt.show()

## 7. Sensitivity Analysis

In [ ]:
def compare_at_different_costs(cost_levels: List[float], n_paths: int = 3000) -> pd.DataFrame:
    """Compare deep hedging vs BS at different transaction cost levels."""
    results = []
    
    for cost in cost_levels:
        print(f"Training with {cost*10000:.0f}bps transaction costs...")
        
        hedger = DeepHedger(
            S0=100, K=100, T=0.25, r=0.05, sigma=0.2,
            n_steps=63, transaction_cost=cost
        )
        
        # Quick training
        for _ in range(30):
            hedger.train_epoch(n_paths=3000, batch_size=256)
        
        # Evaluate
        eval_results = hedger.evaluate(n_paths=n_paths)
        
        results.append({
            'Cost_bps': cost * 10000,
            'Deep_Mean_PnL': np.mean(eval_results['deep_pnl']),
            'Deep_Std_PnL': np.std(eval_results['deep_pnl']),
            'BS_Mean_PnL': np.mean(eval_results['bs_pnl']),
            'BS_Std_PnL': np.std(eval_results['bs_pnl']),
            'Deep_Sharpe': np.mean(eval_results['deep_pnl']) / np.std(eval_results['deep_pnl']) if np.std(eval_results['deep_pnl']) > 0 else 0,
            'BS_Sharpe': np.mean(eval_results['bs_pnl']) / np.std(eval_results['bs_pnl']) if np.std(eval_results['bs_pnl']) > 0 else 0
        })
    
    return pd.DataFrame(results)


# Run sensitivity analysis
cost_levels = [0.0, 0.0005, 0.001, 0.002, 0.005]
sensitivity_df = compare_at_different_costs(cost_levels)
print("\n" + "=" * 70)
print("Sensitivity Analysis Results")
print("=" * 70)
print(sensitivity_df.to_string(index=False))

In [ ]:
# Visualize sensitivity results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Mean P&L comparison
ax1 = axes[0]
ax1.plot(sensitivity_df['Cost_bps'], sensitivity_df['Deep_Mean_PnL'], 'b-o', label='Deep Hedge', linewidth=2)
ax1.plot(sensitivity_df['Cost_bps'], sensitivity_df['BS_Mean_PnL'], 'r--s', label='BS Hedge', linewidth=2)
ax1.axhline(y=0, color='gray', linestyle=':')
ax1.set_xlabel('Transaction Cost (bps)')
ax1.set_ylabel('Mean P&L ($)')
ax1.set_title('Mean P&L vs Transaction Costs')
ax1.legend()

# Std P&L comparison
ax2 = axes[1]
ax2.plot(sensitivity_df['Cost_bps'], sensitivity_df['Deep_Std_PnL'], 'b-o', label='Deep Hedge', linewidth=2)
ax2.plot(sensitivity_df['Cost_bps'], sensitivity_df['BS_Std_PnL'], 'r--s', label='BS Hedge', linewidth=2)
ax2.set_xlabel('Transaction Cost (bps)')
ax2.set_ylabel('Std P&L ($)')
ax2.set_title('P&L Volatility vs Transaction Costs')
ax2.legend()

# Improvement
ax3 = axes[2]
improvement = sensitivity_df['Deep_Mean_PnL'] - sensitivity_df['BS_Mean_PnL']
ax3.bar(sensitivity_df['Cost_bps'], improvement, color='green', alpha=0.7)
ax3.axhline(y=0, color='black', linestyle='-')
ax3.set_xlabel('Transaction Cost (bps)')
ax3.set_ylabel('P&L Improvement ($)')
ax3.set_title('Deep Hedge Improvement over BS')

plt.tight_layout()
plt.show()

## 8. Key Takeaways

### Deep Hedging Advantages

1. **Cost-Aware Learning**: Incorporates transaction costs directly in training
2. **Model-Free**: No need to specify price dynamics (can work with real data)
3. **Adaptive**: Can learn non-linear hedging strategies
4. **Reduced Trading**: Learns to trade less frequently when costs are high

### When Deep Hedging Helps

- High transaction cost environments
- Complex derivatives or exotic options
- Model uncertainty (don't know true dynamics)
- Risk measure customization (CVaR, etc.)

### Limitations

- Requires substantial training data
- Black-box nature (less interpretable)
- May not generalize to unseen market regimes
- Needs careful validation and backtesting

In [ ]:
# Summary
print("\n" + "="*60)
print("DEEP HEDGING SUMMARY")
print("="*60)
print("\nFramework:")
print("  δ_t = f_θ(S_t, τ, other features)")
print("  Learn θ to minimize risk measure of hedging P&L")
print("\nKey Components:")
print("  1. Feature extraction (moneyness, time, etc.)")
print("  2. Neural network for hedge ratio")
print("  3. P&L computation with costs")
print("  4. Loss function (mean-variance, CVaR, etc.)")
print("\nBenefits vs BS Delta:")
print("  - Better performance with transaction costs")
print("  - Learns to trade less frequently")
print("  - Can incorporate market frictions")
print("\nPractical Considerations:")
print("  - Need sufficient training data")
print("  - Validate on out-of-sample data")
print("  - Monitor for regime changes")
print("  - Consider model uncertainty")